### 根据剩余的技术术语及其兴起年，统计各省、各公司的创新能力

In [1]:
import pandas as pd
import numpy as np
import psycopg
import yaml

with open('../../config/config.yaml', mode='r', encoding='utf-8') as f:
    config = yaml.safe_load(f)
    db_config = config['database']

    connection = psycopg.connect(
        dbname=db_config['db_name'],
        user=db_config['user'],
        password=db_config['password'],
        host=db_config['host'],
        port=db_config['port']
    )

tech_noun_phrase_df = pd.read_csv(r'../../output/current_tech_noun_phrases_emergence_year.txt', sep='\t', header=None)
tech_noun_phrase_df.columns = ['tech_noun_phrase', 'emergence_year']
tech_noun_phrase_inverse_index_df = pd.read_csv(r'../../output/current_tech_noun_phrases_inverse_index.txt', sep='\t', header=None)
tech_noun_phrase_inverse_index_df.columns = ['tech_noun_phrase', 'uuid']
valid_uuid_list = list(tech_noun_phrase_inverse_index_df['uuid'].unique())
uuid_list_str = ", ".join([f"'{uuid}'" for uuid in valid_uuid_list])

query = f"""
SELECT 
    t1.uuid, 
    t1.appl_year,
    t2.name AS applicant_name,
    t2.province,
    t2.city
FROM 
    cnipa_appl AS t1
JOIN
    cnipa_appl_applicant AS t2
ON
    t1.uuid = t2.uuid
WHERE 
    t1.patent_type='发明' 
    and CAST(t1.appl_year AS INT) BETWEEN 2014 AND 2023 
    and t1.appl_id is not null
    and t1.uuid IN ({uuid_list_str})
"""

patent_info_df = pd.read_sql(query, connection)

/tmp/ipykernel_19244/4209270824.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  patent_info_df = pd.read_sql(query, connection)


In [13]:
tech_noun_phrase_patent_df = pd.merge(tech_noun_phrase_inverse_index_df, tech_noun_phrase_df, on='tech_noun_phrase', how='inner')
tech_noun_phrase_patent_df = pd.merge(tech_noun_phrase_patent_df, patent_info_df, on='uuid', how='inner')

# 清除emergence_year为空的数据
tech_noun_phrase_patent_df = tech_noun_phrase_patent_df.dropna(subset=['emergence_year'])
# 清除appl_year - emergence_year > 3的数据
tech_noun_phrase_patent_df['appl_year'] = tech_noun_phrase_patent_df['appl_year'].astype(int)
tech_noun_phrase_patent_df['emergence_year'] = tech_noun_phrase_patent_df['emergence_year'].astype(int)
tech_noun_phrase_patent_df = tech_noun_phrase_patent_df[tech_noun_phrase_patent_df['appl_year'] - tech_noun_phrase_patent_df['emergence_year'] <= 3]
# 清除省份为空的数据
tech_noun_phrase_patent_df = tech_noun_phrase_patent_df.dropna(subset=['province'])
# 清除省份为0的数据
tech_noun_phrase_patent_df = tech_noun_phrase_patent_df[tech_noun_phrase_patent_df['province'] != '0']

In [22]:
tech_noun_phrase_inverse_index_df.groupby('tech_noun_phrase').size().sort_values(ascending=False).head(10)

tech_noun_phrase
特征工程      111
联邦学习       93
窄带物联网      81
残差神经网络     69
虚拟现实头戴     61
词嵌入        53
分布式账本      52
折叠屏        45
曲面屏        43
无人机飞行器     35
dtype: int64

#### 计算省级创新能力

专利数量

In [15]:
province_patent_count = tech_noun_phrase_patent_df.groupby('province').size().reset_index(name='patent_count')
province_patent_count = province_patent_count.sort_values(by='patent_count', ascending=False)
province_patent_count

,province,patent_count
10,广东省,168
2,北京市,132
13,江苏省,72
17,浙江省,57
0,上海市,55
8,山东省,41
18,湖北省,28
5,四川省,25
21,福建省,21
7,安徽省,21


#### 计算企业创新能力

In [18]:
company_patent_count = tech_noun_phrase_patent_df.groupby('applicant_name').size().reset_index(name='patent_count')
company_patent_count = company_patent_count.sort_values(by='patent_count', ascending=False)
company_patent_count.head(10)

,applicant_name,patent_count
330,深圳前海微众银行股份有限公司,18
316,浙江大学,13
220,平安科技（深圳）有限公司,13
385,维沃移动通信有限公司,12
152,南京邮电大学,10
287,武汉大学,10
374,电子科技大学,9
435,重庆邮电大学,9
89,努比亚技术有限公司,8
105,北京小鸟看看科技有限公司,7
